## ✅ ขั้นตอนที่ 1: ติดตั้งไลบรารีและโหลดเอกสาร

In [1]:
%pip install nest_asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# ติดตั้งไลบรารีที่จำเป็น
!pip install -q llama-index chromadb nest-asyncio python-docx
!pip install -q ngrok==1.3.0
!pip install -q uvicorn==0.29.0
!pip install -q fastapi==0.111.0
!pip install -q loguru==0.7.2
!pip install -q gdown
!pip install -q docx2txt
%pip install -Uq llama-index-embeddings-huggingface
%pip install -Uq chromadb
%pip install -Uq llama-index-vector-stores-chroma
%pip install -q llama-index-llms-openrouter

# สร้างโฟลเดอร์เก็บไฟล์
!mkdir -p ./data

file_id = "1NOzM7fEtOVGYQNj52Tp36GdSSzZS3U2h"
destination = "./data/1.pdf"

!gdown --id {file_id} -O {destination}


# ดาวน์โหลดเอกสารตัวอย่างสำหรับสร้าง index
!curl -L -o ./data/cv.docx "https://raw.githubusercontent.com/9meo/llm-training/refs/heads/main/material/cv_thai_sample.docx"
!curl -L -o ./data/llm_basics.txt "https://raw.githubusercontent.com/9meo/llm-training/refs/heads/main/material/llm_basics.txt"
!curl -L -o ./data/elephant.txt "https://raw.githubusercontent.com/9meo/llm-training/refs/heads/main/material/thai_elephant_knowledge.txt"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.8 MB/s eta 0:00:00
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1NOzM7fEtOVGYQNj52Tp36GdSSzZS3U2h
To: /content/data/1.pdf
100% 3.86M/3.86M [00:00<00:00, 12.4MB/s]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current


## ✅ ขั้นตอนที่ 2: โหลดเอกสาร และสร้าง Embedding + Index

In [6]:
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# กำหนด LLM ที่จะใช้ (สามารถเปลี่ยน model ได้ตามต้องการ)
llm = OpenRouter(
   api_key="",  # 🔑 ใส่ OpenRouter API Key ที่นี่
    max_tokens=256,
    context_window=4096,
    model="google/gemma-3-27b-it:free",
)




In [7]:
message = ChatMessage(role="user", content="เล่นตลกให้ดูหน่อย อย่างสีเหลือง=เยลโล่  มะม่วง=แมงโก้ มีด=")
resp = llm.chat([message])
print(resp)

assistant: แน่นอน! เล่นตามเลยนะ

สีเหลือง = เยลโล่
มะม่วง = แมงโก้
มีด = ไนฟ์

แล้ว...

ประเทศไทย = ไทยแลนด์! 🇹🇭

เป็นไงบ้าง? สนุกไหม? 😄



In [8]:
# โหลดเอกสารจากโฟลเดอร์ ./data
docs = SimpleDirectoryReader(input_dir="./data").load_data()

In [9]:
import pprint
pprint.pprint(docs)

[Document(id_='26bc04ae-b98a-41b6-abb5-8221713b345e', embedding=None, metadata={'page_label': '1', 'file_name': '1.pdf', 'file_path': '/content/data/1.pdf', 'file_type': 'application/pdf', 'file_size': 3857399, 'creation_date': '2025-03-26', 'last_modified_date': '2025-03-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='ปัญญาประดิษฐ์เพื่อการประเมินการกัดกร่อน\nในบรรยากาศและแผนที่การกัดกร่อน:\nความยั่งยืนและความปลอดภัยของโครงสร้างเหล็กกล้า\nAI in Atmospheric Corrosion Assessment and Corroison Map:\nSteel Infrastructure Safety and Sustainability\n การกัดกร่อนของเหล็กกล้าโครงสร้างใน\nบรรยากาศขึ้นอยู่กับปัจจัยสภาพอากาศและ\nสภาพแวดล้อม เอ็

In [10]:
docs[0].__dict__

{'id_': '26bc04ae-b98a-41b6-abb5-8221713b345e',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': '1.pdf',
  'file_path': '/content/data/1.pdf',
  'file_type': 'application/pdf',
  'file_size': 3857399,
  'creation_date': '2025-03-26',
  'last_modified_date': '2025-03-19'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='ปัญญาประดิษฐ์เพื่อการประเมินการกัดกร่อน\nในบรรยากาศและแผนที่การกัดกร่อน:\nความยั่งยืนและความปลอดภัยของโครงสร้างเหล็กกล้า\nAI in Atmospheric Corrosion Assessment and Corroison Map:\nSteel Infrastructure Safety and Sustainability\n การกัดกร่อนของเหล็กกล้าโครงสร้าง

In [11]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# ตั้งค่าให้แยกเอกสารเป็น chunk ย่อย ๆ
text_splitter = SentenceSplitter(
    separator=" ",       # ใช้เว้นวรรคเป็นตัวแยก
    chunk_size=1024,     # ขนาดของแต่ละ chunk (จำนวนตัวอักษร)
    chunk_overlap=128    # ความซ้อนของข้อความระหว่างแต่ละ chunk
)

# สร้าง ingestion pipeline ที่ประกอบด้วย text_splitter
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
    ]
)

# รัน pipeline กับเอกสารที่โหลดไว้ เพื่อแยกเป็น chunks
nodes = pipeline.run(
    documents=docs,
    in_place=True,       # แก้ไขเอกสารเดิมแทนที่จะสร้างใหม่
    show_progress=True,  # แสดง progress bar ขณะรัน
)


Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
nodes[2].__dict__

{'id_': '15bbd20d-9003-44fb-a29c-a60aa747a868',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': '1.pdf',
  'file_path': '/content/data/1.pdf',
  'file_type': 'application/pdf',
  'file_size': 3857399,
  'creation_date': '2025-03-26',
  'last_modified_date': '2025-03-19'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='26bc04ae-b98a-41b6-abb5-8221713b345e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '1.pdf', 'file_path': '/content/data/1.pdf', 'file_type': 'application/pdf', 'file_size': 3857399, 'creation_date': '2025-03-26', 'last_modified_date': '2025-03-19'}, hash='15f2b4952c5a1f9e8a056601aa9dae77c9d3688a1f8d74f9128

In [13]:
# ใช้โมเดล HuggingFace สำหรับสร้าง embedding
hf_embeddings = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
# เริ่มต้น client ของ ChromaDB และสร้าง collection สำหรับเก็บเวกเตอร์
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("rag_demo")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# สร้าง storage context สำหรับบันทึกข้อมูล
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# สร้าง index จากเอกสาร โดยใช้ embedding ที่เรากำหนด
index = VectorStoreIndex(
    nodes=nodes, storage_context=storage_context, embed_model=hf_embeddings,show_progress=True,  # แสดง progress bar ขณะรัน
)

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

In [15]:
# สร้าง query engine สำหรับถามคำถามจาก index
query_engine = index.as_query_engine(llm=llm)

In [16]:
# ตั้งคำถามที่ต้องการ
question = "ช้างไทยมีลักษณะอย่างไร"

# ใช้ query_engine ในการค้นหาคำตอบจากข้อมูลใน index
response = query_engine.query(question)

# แสดงผลลัพธ์
print("คำถาม:", question)
print("คำตอบ:", response)


คำถาม: ช้างไทยมีลักษณะอย่างไร
คำตอบ: 
ช้างไทยมีความสูงประมาณ 2.5 - 3.2 เมตร และมีน้ำหนัก 2,700 - 4,500 กิโลกรัม มีอายุขัยอยู่ที่ 60 - 80 ปี มีหัวเป็นโค้ง คอหนา และหูขนาดเล็กกว่าช้างแอฟริกา พวกมันกินพืช หญ้า ใบไม้ ผลไม้ และเปลือกไม้ ได้ถึง 150 กิโลกรัมต่อวัน


In [17]:
# แสดงแหล่งอ้างอิง (เช่น ชื่อไฟล์ หรือเนื้อหาบางส่วนที่เกี่ยวข้อง)
for node in response.source_nodes:
    print("--- แหล่งข้อมูลที่ใช้ ---")
    print(node.node.text[:300])  # แสดงเฉพาะ 300 ตัวอักษรแรก


--- แหล่งข้อมูลที่ใช้ ---
## ความรู้เกี่ยวกับช้างไทย

### 1. ช้างไทยคืออะไร?
ช้างไทย (Asian Elephant - Elephas maximus) เป็นสัตว์สัญลักษณ์ของประเทศไทยและมีบทบาทสำคัญทั้งในวัฒนธรรม ประวัติศาสตร์ และเศรษฐกิจของประเทศ โดยช้างไทยจัดอยู่ในกลุ่มช้างเอเชีย ซึ่งมีขนาดเล็กกว่าช้างแอฟริกา และมีลักษณะเฉพาะที่แตกต่างกัน เช่น หัวเป็นโค้ง


## ✅ ขั้นตอนที่ 3: สร้าง FastAPI endpoint สำหรับทำการค้นหา

In [18]:
NGROK_AUTH_TOKEN = ""
APPLICATION_PORT = 5000
NGROK_EDGE = "edge:edghts_"

In [20]:
import os
import nest_asyncio
import threading
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok

# 🔐 ใส่ ngrok token ตรงนี้ (จาก https://dashboard.ngrok.com)
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# สร้าง FastAPI
app = FastAPI()

class QueryRequest(BaseModel):
    query: str

@app.post("/search")
def search(request: QueryRequest):
    response = query_engine.query(request.query)
    return {"query": request.query, "response": str(response)}

# รัน FastAPI บน Colab
nest_asyncio.apply()

def run():
    import uvicorn
    from uvicorn import Config, Server
    config = Config(app=app, host="0.0.0.0", port=5000, log_level="info")
    server = Server(config)

    import asyncio
    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

# รันใน thread
threading.Thread(target=run).start()

# เปิด ngrok
public_url = ngrok.connect(5000)
print(f"🔗 FastAPI endpoint: {public_url}/search")


INFO:     Started server process [9690]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


🔗 FastAPI endpoint: NgrokTunnel: "https://87d3-35-231-44-44.ngrok-free.app" -> "http://localhost:5000"/search


## ✅ วิธีทดลองใช้งาน API

In [23]:
# ตัวอย่างคำสั่ง curl สำหรับเรียกใช้งาน API ที่ได้จาก ngrok
# อย่าลืมแทนที่ URL ด้านล่างด้วยลิงก์ที่ได้จาก ngrok

!curl -X POST https://87d3-35-231-44-44.ngrok-free.app/search \
  -H "Content-Type: application/json" \
  -d '{"query": "ช้างไทยกินอะไร"}'


INFO:     35.231.44.44:0 - "POST /search HTTP/1.1" 200 OK
{"query":"ช้างไทยกินอะไร","response":"\nช้างไทยกินพืช หญ้า ใบไม้ ผลไม้ และเปลือกไม้ ได้มากถึง 150 กิโลกรัมต่อวัน"}